## Runable Sequence 链

In [3]:
# Runable Sequence 链
from langchain_fireworks import ChatFireworks
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()
# Prompt接收字典对象，返回PromptValue对象
prompt = ChatPromptTemplate.from_template("讲一个关于{topic}的笑话。")
# ChatModel接收PromptValue对象，返回ChatMessage对象
chat = ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct")
# 注意Runnable Sequence 链中各结点的输入和输出
chain = prompt|chat
# 注意Runnabel Sequence 链的输入和输出
chain.invoke({"topic":"狼"})

AIMessage(content='Why are wolves always asking for directions?\n\nBecause they have "fleas" for navigation! (sounds like "GPS")\n\nI hope you found this joke about a wolf\'s sense of direction amusing. If you need more laughter or have any other requests, feel free to ask!', response_metadata={'token_usage': {'prompt_tokens': 20, 'total_tokens': 86, 'completion_tokens': 66}, 'model_name': 'accounts/fireworks/models/mixtral-8x7b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-26013abc-859a-4f28-aca3-24d825fe3daf-0')

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
import os
from langchain_fireworks import ChatFireworks
from dotenv import load_dotenv
load_dotenv()
# 使用Mistral-7b
llm = ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct")
# 定义笑话链
joke_chain = ChatPromptTemplate.from_template("讲一句关于{topic}的笑话")|llm
# 定义诗歌链
poem_chain = ChatPromptTemplate.from_template("写一首关于{topic}的短诗")|llm
# 通过RunnableParallel执行两个调用链
map_chain = RunnableParallel(joke=joke_chain,poem=poem_chain)
map_chain.invoke({"topic":"小白兔"})

{'joke': AIMessage(content='小白兔为什么总是开心？因为它知道它不会被猎杀，因为它是“兔”斯基！(Rabbit-Ski)\n :)\n\n(请注意，这是一句 humor/梗话，不是事实。)', response_metadata={'token_usage': {'prompt_tokens': 22, 'total_tokens': 97, 'completion_tokens': 75}, 'model_name': 'accounts/fireworks/models/mixtral-8x7b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ca97496-53a8-4cbb-ad12-9ab942336362-0'),
 'poem': AIMessage(content="小白兔在草地上，\nPlaying in the green, green grass,\nSoft and fluffy, full of fun,\nHopping here and there, so fast.\n\nWith twinkling eyes and pink little nose,\nNibbling on clover, carrots too,\nIn the sunshine, basking,\nLiving life, simple and true.\n\nDreaming under the moon's soft glow,\nIn the world of nature, free to go,\nThe little bunny, full of woe,\nA symbol of peace, we should know.", response_metadata={'token_usage': {'prompt_tokens': 24, 'total_tokens': 157, 'completion_tokens': 133}, 'model_name': 'accounts/fireworks/models/mixtral-8x7b-instruct', 'system_fing

In [4]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_fireworks import ChatFireworks

def length_function(text):
    return len(text)
def multiple_length_function(_dict):
    return len(_dict["text1"]) * len(_dict["text2"])
prompt = ChatPromptTemplate.from_template("What is {a}+{b}")
model = ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct")
chain = {"a": itemgetter("foo")|RunnableLambda(length_function),
         "b": {"text1":itemgetter("foo"), "text2":itemgetter("bar")} | RunnableLambda(multiple_length_function)}|prompt|model
chain.invoke({"foo":"bar","bar":"gah"})

AIMessage(content="The sum of 3 and 9 is 12. Is there anything else you would like to know? I'm here to help with any questions you have. Just let me know!", response_metadata={'token_usage': {'prompt_tokens': 14, 'total_tokens': 55, 'completion_tokens': 41}, 'model_name': 'accounts/fireworks/models/mixtral-8x7b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-faae7829-ee8e-4ed7-850b-d82920093ea8-0')

In [12]:
# 设计求职助手链
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate

def label_to_response(label):
    return "classify response"

def question_retrieval(text):
    return "retrieval response"

# 定义RunnableLambda对象，接收一个字符串，返回字符串
question_classify_chain = RunnableLambda(label_to_response)
print("question_classify_chain 类型: ", type(question_classify_chain))
# 定义RunnableLambda对象，接收一个字符串，返回字符串
question_retrieval_chain = RunnableLambda(question_retrieval)
print("question_retrieval_chain 类型: ", type(question_retrieval_chain))

# 定义RunnableParallel对象，接收多个Runnable对象，返回词典
parallel_chain = RunnableParallel(question_classify_response=question_classify_chain,
                                  question_retrieval_response=question_retrieval)
print("parallel_chain 类型: ", type(parallel_chain))
# 定义ChatPrompt对象，接收词典，返回PromptValue对象
prompt = ChatPromptTemplate.from_template("合并两个链的结果{question_classify_response}和{question_retrieval_response}。")
print("prompt 类型: ", type(prompt))
# 总链为并行链 | 系统提示
final_chain = parallel_chain | prompt
final_chain.invoke("你好！")


question_classify_chain 类型:  <class 'langchain_core.runnables.base.RunnableLambda'>
question_retrieval_chain 类型:  <class 'langchain_core.runnables.base.RunnableLambda'>
parallel_chain 类型:  <class 'langchain_core.runnables.base.RunnableParallel'>
prompt 类型:  <class 'langchain_core.prompts.chat.ChatPromptTemplate'>


ChatPromptValue(messages=[HumanMessage(content='合并两个链的结果classify response和retrieval response。')])